In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
import os

dataset_path = "/content/drive/MyDrive/chestXRay"

for data in os.listdir(dataset_path):
  print(data)

if os.path.exists(dataset_path):
    print("Dataset found!")
    print(os.listdir(dataset_path))
else:
    print("Dataset not found. Make sure you added a shortcut in My Drive.")

In [ ]:
import shutil
import random
import numpy as np
import pandas as pd
import cv2
import skimage
import matplotlib.pyplot as plt
import skimage.segmentation
import seaborn as sns
%matplotlib inline
plt.style.use('ggplot')

In [ ]:
def get_data(data_dir):
    labels = ['PNEUMONIA', 'NORMAL']
    img_size = 128
    data = []
    for label in labels:
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        if not os.path.exists(path):
            print(f"Warning: {path} does not exist! Skipping...")
            continue
        for img in os.listdir(path):
            if not img.lower().endswith(('.png', '.jpg', '.jpeg')):
                continue
            try:
                img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                resized_arr = cv2.resize(img_arr, (img_size, img_size))
                data.append([resized_arr, class_num])
            except Exception as e:
                print(f"Error loading image {img}: {e}")
    if len(data) == 0:
        print(f"Error: No data found in {data_dir}!")
        return None, None
    images = np.array([entry[0] for entry in data])
    labels = np.array([entry[1] for entry in data])
    return images, labels

In [ ]:
train_images, train_labels = get_data("/content/drive/My Drive/chestXRay/chest_xray/train")
test_images, test_labels = get_data("/content/drive/My Drive/chestXRay/chest_xray/test")
val_images, val_labels = get_data("/content/drive/My Drive/chestXRay/chest_xray/val")


if train_images is not None:
    print(train_images.shape, train_labels.shape)
    print(test_images.shape, test_labels.shape)
    print(val_images.shape, val_labels.shape)

In [ ]:
pneumonia = os.listdir("/content/drive/My Drive/chestXRay/chest_xray/train/PNEUMONIA")
penomina_dir = "/content/drive/My Drive/chestXRay/chest_xray/train/PNEUMONIA"

plt.figure(figsize=(20,10))

for i in range(9):
    plt.subplot(3,3, i+1)
    img = plt.imread(os.path.join(penomina_dir, pneumonia[i]))
    plt.imshow(img, cmap='gray')
    plt.axis("off")
    plt.title("Pneumonia X-ray")
plt.tight_layout()

In [ ]:
normal = os.listdir("/content/drive/My Drive/chestXRay/chest_xray/train/NORMAL")
normal_dir = "/content/drive/My Drive/chestXRay/chest_xray/train/NORMAL"

plt.figure(figsize=(20,10))

for i in range(9):
    plt.subplot(3,3, i+1)
    img = plt.imread(os.path.join(normal_dir, normal[i]))
    plt.imshow(img, cmap='gray')
    plt.axis("off")
    plt.title("normal X-ray")
plt.tight_layout()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

listx = []
for i in range(len(train_labels)):
    if train_labels[i] == 0:
        listx.append("Pneumonia")
    else:
        listx.append("Normal")

sns.countplot(x=listx, palette="Set2")
plt.show()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1. / 255,
                  horizontal_flip=0.4,
                  vertical_flip=0.4,
                  rotation_range=40,
                  shear_range=0.2,
                  width_shift_range=0.4,
                  height_shift_range=0.4,
                  fill_mode="nearest")
valid_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory("/content/drive/My Drive/chestXRay/chest_xray/train",
                                 batch_size = 32,
                                 target_size=(128,128),
                                 class_mode = 'categorical',
                                 shuffle=True,
                                 seed = 42,
                                 color_mode = 'rgb')
valid_generator = valid_datagen.flow_from_directory("/content/drive/My Drive/chestXRay/chest_xray/val",
                                 batch_size = 32,
                                 target_size=(128,128),
                                 class_mode = 'categorical',
                                 shuffle=True,
                                 seed = 42,
                                 color_mode = 'rgb')
test_generator = test_datagen.flow_from_directory("/content/drive/My Drive/chestXRay/chest_xray/test",
                                 batch_size = 32,
                                 target_size=(128,128),
                                 class_mode = 'categorical',
                                 shuffle=True,
                                 seed = 42,
                                 color_mode = 'rgb')